In [1]:
import { BufferWindowMemory } from "langchain/memory"
import { ChatOpenAI } from "@langchain/openai"
import { BufferMemory, ConversationSummaryMemory } from "langchain/memory"
import { ConversationChain } from "langchain/chains"
import { PromptTemplate } from "@langchain/core/prompts"
import { ConversationSummaryBufferMemory } from "langchain/memory"
import { ChatPromptTemplate } from "@langchain/core/prompts"
import {
  RunnableSequence,
  RunnablePassthrough,
} from "@langchain/core/runnables"
import { StringOutputParser  } from "@langchain/core/output_parsers";


In [2]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 1.5,
  modelName: "deepseek-v3",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}

## 在 LCEL 中集成 memory

In [3]:
const chatModel = new ChatOpenAI(chatOptions)
const memory = new BufferMemory()

const TEMPLATE = `
你是一个乐于助人的 ai 助手。尽你所能回答所有问题。

这是跟人类沟通的聊天历史:
{history}

据此回答人类的问题:
{input}
`
const prompt = ChatPromptTemplate.fromTemplate(TEMPLATE);

In [4]:
console.log("prompt", prompt)

prompt ChatPromptTemplate {
  lc_serializable: true,
  lc_kwargs: {
    inputVariables: [ "history", "input" ],
    promptMessages: [
      HumanMessagePromptTemplate {
        lc_serializable: true,
        lc_kwargs: { prompt: [PromptTemplate] },
        lc_runnable: true,
        name: undefined,
        lc_namespace: [ "langchain_core", "prompts", "chat" ],
        inputVariables: [ "history", "input" ],
        additionalOptions: {},
        prompt: PromptTemplate {
          lc_serializable: true,
          lc_kwargs: [Object],
          lc_runnable: true,
          name: undefined,
          lc_namespace: [Array],
          inputVariables: [Array],
          outputParser: undefined,
          partialVariables: undefined,
          templateFormat: "f-string",
          template: "\n" +
            "你是一个乐于助人的 ai 助手。尽你所能回答所有问题。\n" +
            "\n" +
            "这是跟人类沟通的聊天历史:\n" +
            "{history}\n" +
            "\n" +
            "据此回答人类的问题:\n" +
            "{input}\n",

In [5]:
let tempInput = ""

const chain = RunnableSequence.from([
  {
    input: new RunnablePassthrough(),
    memoryObject: async (input) => {
      const history = await memory.loadMemoryVariables({
        input,
      })

      tempInput = input
      return history
    },
  },
  RunnablePassthrough.assign({
    history: (input) => input.memoryObject.history,
  }),
  prompt,
  chatModel,
  new StringOutputParser(),
  new RunnablePassthrough({
    func: async (output) => {
      await memory.saveContext({
        input: tempInput,
      }),
        {
          output,
        }
    },
  }),
])


## 实现自定义的 chat history

In [6]:
import { BaseListChatMessageHistory } from "@langchain/core/chat_history"
import {
  BaseMessage,
  StoredMessage,
  mapChatMessagesToStoredMessages,
  mapStoredMessagesToChatMessages,
} from "@langchain/core/messages"
import fs from "node:fs"
import path from "node:path"

export interface JSONChatHistoryInput {
  sessionId: string
  dir: string
}

export class JSONChatHistory extends BaseListChatMessageHistory {
  lc_namespace = ["langchain", "stores", "message"]

  sessionId: string
  dir: string

  constructor(fields: JSONChatHistoryInput) {
    super(fields)
    this.sessionId = fields.sessionId
    this.dir = fields.dir
  }

  async getMessages(): Promise<BaseMessage[]> {
    const filePath = path.join(this.dir, `${this.sessionId}.json`)
    try {
      if (!fs.existsSync(filePath)) {
        this.saveMessagesToFile([])
        return []
      }

      const data = fs.readFileSync(filePath, { encoding: "utf-8" })
      const storedMessages = JSON.parse(data) as StoredMessage[]
      return mapStoredMessagesToChatMessages(storedMessages)
    } catch (e) {
      console.log(`Failed to read chat history from ${filePath}:`, e)
      return []
    }
  }

  async addMessage(message: BaseMessage): Promise<void> {
    const messages = await this.getMessages();
    messages.push(message);
    await this.saveMessagesToFile(messages);
  }

  async addMessages(messages: BaseMessage[]): Promise<void> {
    const existingMessages = await this.getMessages();
    const allMessages = existingMessages.concat(messages);
    await this.saveMessagesToFile(allMessages);
  }

  async clear(): Promise<void> {
    const filePath = path.join(this.dir, `${this.sessionId}.json`);
    try {
      fs.unlinkSync(filePath);
    } catch (error) {
      console.error(`Failed to clear chat history from ${filePath}`, error);
    }
  }

  private async saveMessagesToFile(messages: BaseMessage[]): Promise<void> {
    const filePath = path.join(this.dir, `${this.sessionId}.json`);
    const serializedMessages = mapChatMessagesToStoredMessages(messages);
    try {
      fs.writeFileSync(filePath, JSON.stringify(serializedMessages, null, 2), {
        encoding: "utf-8",
      });
    } catch (error) {
      console.error(`Failed to save chat history to ${filePath}`, error);
    }
  }
}


In [7]:
import { AIMessage, HumanMessage } from "@langchain/core/messages";

const history = new JSONChatHistory({
    dir: "chat_data",
    sessionId: "test"
})


await history.addMessages([
  new HumanMessage("Hi, 我叫小明"),
  new AIMessage("你好"),
]);

const messages = await history.getMessages();
console.log(messages)

Failed to read chat history from chat_data\test.json: SyntaxError: Unexpected end of JSON input
    at JSON.parse (<anonymous>)
    at JSONChatHistory.getMessages (<anonymous>:28:35)
    at JSONChatHistory.addMessages (<anonymous>:41:41)
    at <anonymous>:6:15
    at eventLoopTick (ext:core/01_core.js:177:7)
[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]
[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_

In [8]:
const chatModel = new ChatOpenAI(chatOptions);
const memory = new BufferMemory({
    chatHistory: history
});
const chain = new ConversationChain({ llm: chatModel, memory: memory });
const res1 = await chain.call({ input: "我叫什么？" });

In [9]:
console.log("res1", res1)

res1 { response: "你刚刚说：“我叫小明”,所以你的名字是小明。很高兴认识你，小明！😊" }


In [11]:
const messages = await history.getMessages()
console.log('message', messages)

message [
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "Hi, 我叫小明",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "Hi, 我叫小明",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: { content: "你好", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你好",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "我叫什么？", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "我叫什么？",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "你刚刚说：“我叫小明”,所以你的名字是小明。很高兴认识你，小明！😊",
      additional_kwar